<a href="https://colab.research.google.com/github/Prakuljn/Python/blob/main/Vector_db(chromadb).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb google-generativeai sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 10.6 MB/s

In [11]:
import google.generativeai as genai
import os
from google.colab import userdata
# userdata.get('Google_API_KEY')
os.environ["Google_API_KEY"] = userdata.get('Google_API_KEY')
genai.configure(api_key = os.getenv("Google_API_KEY"))

In [12]:
import chromadb
chroma_client = chromadb.PersistentClient(path = "./chroma_db")
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="gemini_embeddings")

In [13]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def generate_embedding(text):
  return embedding_model.encode(text).tolist()

In [14]:
documents = [
    {"id": "1", "text": "What is machine learning?"},
    {"id": "2", "text": "Explain deep learning and its applications."},
    {"id": "3", "text": "What are transformers in NLP?"}
]

for doc in documents:
    embedding = generate_embedding(doc["text"])
    collection.add(
        ids=[doc["id"]],
        embeddings=[embedding],
        metadatas=[{"text": doc["text"]}]
    )

print("Documents inserted successfully!")


Documents inserted successfully!


In [15]:
query_text = "Tell me What is ml"
query_embedding = generate_embedding(query_text)

results = collection.query(
    query_embeddings = [query_embedding],

    n_results =2
)

for i , result in enumerate(results["metadatas"][0]):
  print(f"Result {i+1}: {result['text']}")

Result 1: What is machine learning?
Result 2: Explain deep learning and its applications.


In [16]:
def get_gemini_response(prompt):
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text

# Generate response based on retrieved data
if results["metadatas"][0]:
    context = results["metadatas"][0][0]["text"]
    response = get_gemini_response(f"Explain in detail: {context}")
    print("\nAI Response:\n", response)
else:
    print("No relevant results found.")



AI Response:
 ## Machine Learning: A Deep Dive

Machine learning (ML) is a subfield of artificial intelligence (AI) that focuses on enabling computer systems to learn from data **without being explicitly programmed**.  Instead of writing specific code to solve a problem, you feed the system data and algorithms that allow it to learn patterns, make predictions, and improve its performance over time.  Think of it as teaching a computer to "figure things out" on its own.

Here's a breakdown of the key concepts:

**1. Learning from Data:**

*   **Data is King:**  Machine learning algorithms thrive on data. The more data, the better the potential for accurate and reliable learning. This data can come in various forms:
    *   **Numerical Data:**  Numbers, like sensor readings, stock prices, or customer ages.
    *   **Categorical Data:**  Labels, like colors (red, blue, green), types of fruit (apple, banana, orange), or customer demographics (male, female).
    *   **Text Data:**  Document